In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import pandas_datareader as web

from collections import Counter
import pickle

%run functions.ipynb

In [2]:
#Import models
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import tensorflow as tf
from tensorflow import keras

In [3]:
from bokeh.plotting import figure,output_file, show
from bokeh.layouts import column
from bokeh.embed import components
from bokeh.models import ColumnDataSource, DatetimeTickFormatter, Legend, Title, Arrow, HoverTool
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [4]:
%run functions.ipynb

In [5]:
def create_pred_df(ticker):

    pred_start = pd.datetime.today() - pd.to_timedelta(100, unit='day') 
    pred_end = pd.datetime.today()
    
    columns = ['Returns', 'Unemployment Rate', 'Inflation Rate', 'Misery Index',
       'local_price', 'Bullish', 'Neutral', 'Bearish', 'Bull-Bear Spread',
       'EMA12_pctchg', 'RSI', 'Pct_Chg', 'MACD_sentiment', 'Upper_BB_pctchg',
       'Lower_BB_pctchg', 'EMA_sentiment', 'Volume_sentiment']
    
    df = web.DataReader(ticker, 'yahoo', pred_start, pred_end)

    adv_stock_indicators(df)

    df['MACD_sentiment'] = list(map(classify_bullish, df['MACD'], df['MACD_Signal']))
    df['EMA_sentiment'] = list(map(classify_bullish, df['EMA26'], df['EMA12']))
    df['Volume_sentiment']  = list(map(classify_bullish, df['Dollar_volume'], df['Rolling_Dollar_volume']))
    
    alternative_data = pd.read_csv('data/cleaned_alternative_data.csv', index_col='index')
    alternative_data.drop('Unnamed: 0', axis=1, inplace=True)
    
    full_df = alternative_data.join(df).dropna()

    full_df['svm_buy_predictions'] = svm_cv.predict(full_df[columns])
    full_df['logreg_buy_predictions'] = logreg_cv.predict(full_df[columns])
    full_df['rf_buy_predictions'] = rf_cv.predict(full_df[columns])
    full_df['xgboost_buy_predictions'] = xgb_cv.predict(full_df[columns])
    full_df['ann_buy_predictions'] = np.argmax(ann.predict(full_df[columns]), axis=-1)
    
    full_df['buy_predictions'] = full_df[['svm_buy_predictions',
                                            'logreg_buy_predictions',
                                            'rf_buy_predictions', 
                                            'xgboost_buy_predictions',
                                            'ann_buy_predictions'
                                            ]].mode(axis=1)

    return full_df

In [6]:
#Bull
svm_cv = pickle.load(open("models/svm_cv_best.pickle","rb"))
logreg_cv = pickle.load(open("models/logreg_cv_best.pickle","rb"))
rf_cv = pickle.load(open("models/rf_cv_best.pickle","rb"))
xgb_cv = pickle.load(open("models/xgb_cv_best.pickle","rb"))
ann = tf.keras.models.load_model('models/ann.h5')

In [7]:
def predict_ticker(ticker):
    predict_df = create_pred_df(ticker)
    
    df1 = predict_df

    inc = df1['Close'] > df1['Open']
    dec = df1['Open'] > df1['Close']
    w = 12*60*60*1000

    p = figure(title= f'{ticker}'+ ' Chart\n', 
               width=1000, height=475, 
               #x_axis_label='Date', 
               y_axis_label='Price (USD)',
               y_range=(min(df1['Close'])/1.20, max(df1['High'])*1.10))

    #background
    p.background_fill_color = "white"
    p.background_fill_alpha = 0.5

    #gridlines
    p.xgrid.minor_grid_line_color = 'navy'
    p.xgrid.minor_grid_line_alpha = 0.05
    p.ygrid.minor_grid_line_color = 'navy'
    p.ygrid.minor_grid_line_alpha = 0.05

    #title
    p.title.text_font_size = '20pt'
    p.title.align = 'center'
    p.add_layout(Title(
        text="SVM, Logistic Regression, XGBoost, Random Forest, ANN Predictions"
        ), 'above')
    p.add_layout(Title(
        text="Alternative and Technical Model"
        ), 'above')

    #axes
    p.xaxis.axis_label_text_font_size = "13pt"
    p.yaxis.axis_label_text_font_size = "13pt"
    p.xaxis.formatter=DatetimeTickFormatter(days="%m/%d")
    p.xaxis.ticker.desired_num_ticks=10

    #Candlestick
    p.segment(df1.index, df1['High'], df1.index, df1['Low'], color='black')
    p.vbar(df1.index[inc], w, df1['Open'][inc], df1['Close'][inc], fill_color='green', line_color='black')
    p.vbar(df1.index[dec], w, df1['Open'][dec], df1['Close'][dec], fill_color='red', line_color='black')

    #EMA
    e12 = p.line(df1.index, df1['EMA12'], line_color='darkorange', line_dash='dotted', alpha=0.90)
    e26 = p.line(df1.index, df1['EMA26'], line_color='navy', line_dash='dotted', alpha=0.90)

    #BBs
    bb = p.line(df1.index, df1['Upper_BB'], line_color='black', line_dash='solid', alpha=0.50)
    p.line(df1.index, df1['Lower_BB'], line_color='black', line_dash='solid', alpha=0.50)

    #ML Prediction
    buy_ml = p.circle(df1.index, (df1['Low']*df1['buy_predictions']) - (df1['Low'] * 0.05), size=7, color='darkgreen', fill_alpha=0.50)
    #sell_ml = p.circle(df1.index, (df1['High']*df1['sell_predictions']) + (df1['High'] * 0.05), size=7, color='darkred', fill_alpha=0.50)

    #Legend
    legend = Legend(items=[
        ("12-Day Exp Moving Average",   [e12]),
        ("26-Day Exp Moving Average", [e26]),
        ("Bollinger Bands", [bb]),
        ("ML 5-Day Buy Signal", [buy_ml]),
    #    ("ML 5-Day Sell Signal", [sell_ml])
    ], location=(0, 240))
    p.add_layout(legend, 'right')

    #Volume
    bar = figure(
                width=1000, height=100,
                y_axis_label='Volume')
    bar.yaxis.axis_label_text_font_size = "13pt"
    bar.xaxis.formatter=DatetimeTickFormatter()
    bar.xaxis.ticker.desired_num_ticks=10
    bar.vbar(df1.index, top = df1['Volume']/1000000, width=10)

    #RSI 
    p2 = figure(
                width=1000, height=125,
                x_axis_label='Date',
                y_axis_label='RSI')
    p2.yaxis.axis_label_text_font_size = "13pt"
    p2.xaxis.axis_label_text_font_size = "13pt"

    p2.xaxis.formatter=DatetimeTickFormatter()
    p2.xaxis.ticker.desired_num_ticks=10
    p2.line(df1.index, df1['RSI'], line_color='orange')
    p2.line(df1.index, 70, line_color='gray', line_dash='dashed')
    p2.line(df1.index, 30, line_color='gray', line_dash='dashed')

    show(column(p, bar, p2))

In [10]:
predict_ticker('FB')